# Custom Open Images Dataset Download




In [1]:
#Dataset Variable
classes = ['Horse','Sheep','Cattle']
nr_train_images = 1000
nr_val_images = 200

In [2]:
#Prevent output to be displayed
%%capture

# Install Required libraries
!pip install fiftyone

In [3]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create a folder
drive_path = '/content/drive/MyDrive'

# Dataset Paths
import os
HOME                  = os.getcwd()
DATASETS_DIR          = f'{drive_path}/datasets'
DATASET_OI_FMT_DIR    = f'{HOME}/open-images-v7'
DATASET_YOLO_FMT_DIR  = f'{DATASETS_DIR}/OpenImages-LiveStock'
DATASET_YOLO_YAML_DIR = f'{DATASET_YOLO_FMT_DIR}/dataset.yaml'

#Create datasets folder
os.makedirs(DATASETS_DIR, exist_ok=True)

ValueError: mount failed

In [ ]:
#Prevent output to be displayed
%%capture

# Download dataset from OpenImages
import fiftyone as fo
import fiftyone.zoo as foz

# Load the Open Images V7 dataset
train_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    label_types=["detections"],
    classes=classes,
    max_samples=nr_train_images,  # Adjust this number based on your needs
    dataset_dir=DATASET_OI_FMT_DIR,
    splits=("train"),
)

nontrain_dataset = foz.load_zoo_dataset(
    'open-images-v7',
    label_types=["detections"],
    classes=classes,
    max_samples=nr_val_images,  # Adjust this number based on your needs
    dataset_dir=DATASET_OI_FMT_DIR,
    splits=("validation","test"),
)

In [ ]:
#Prevent output to be displayed
%%capture

#Convert Training Dataset
!rm -rf {DATASET_YOLO_FMT_DIR}

train_dataset.export(
        export_dir=DATASET_YOLO_FMT_DIR,
        dataset_type=fo.types.YOLOv5Dataset,
        split='train',
        classes=classes,
)

#Convert Validation and Test Dataset
splits = ['validation','test']
for split in splits:
    split_view = nontrain_dataset.match_tags(split)
    split_view.export(
        export_dir=DATASET_YOLO_FMT_DIR,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        classes=classes,
    )

In [ ]:
# Rename Validation folders
!mv {DATASET_YOLO_FMT_DIR}/images/validation {DATASET_YOLO_FMT_DIR}/images/val
!mv {DATASET_YOLO_FMT_DIR}/labels/validation {DATASET_YOLO_FMT_DIR}/labels/val

!grep -v 'validation:' {DATASET_YOLO_FMT_DIR}/dataset.yaml > {DATASET_YOLO_FMT_DIR}/data_temp.yaml && mv {DATASET_YOLO_FMT_DIR}/data_temp.yaml {DATASET_YOLO_FMT_DIR}/dataset.yaml
!echo -e "val: ../images/val" >> {DATASET_YOLO_FMT_DIR}/dataset.yaml

In [ ]:
# Remove Temporary Downloads
!rm -rf {DATASET_OI_FMT_DIR}

In [ ]:
#Unmount Google Drive
drive.flush_and_unmount()